<a href="https://colab.research.google.com/github/roni1t/electricity-savings-app/blob/main/electricity_savings_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pandas

In [2]:

from google.colab import files
uploaded = files.upload()


Saving meter_23116869_LP_27-04-2025 (1).csv to meter_23116869_LP_27-04-2025 (1).csv


In [3]:

import pandas as pd

# קרא את שם הקובץ שהועלה
import io
file_name = next(iter(uploaded))

# קריאה וניקוי
with open(file_name, 'r', encoding='utf-8', errors='ignore') as f:
    lines = f.readlines()

start_index = next(i for i, line in enumerate(lines) if "תאריך" in line)
df = pd.read_csv(io.StringIO(''.join(lines[start_index:])))
df = df.dropna()
df = df[df['תאריך'].str.contains(r'\d{2}/\d{2}/\d{4}')]
df['datetime'] = pd.to_datetime(df['תאריך'] + ' ' + df['מועד תחילת הפעימה'], format='%d/%m/%Y %H:%M')
df['צריכה בקוט"ש'] = pd.to_numeric(df['צריכה בקוט"ש'], errors='coerce')
df['שעה'] = df['datetime'].dt.hour
df['יום בשבוע'] = df['datetime'].dt.dayofweek
df['חודש'] = df['datetime'].dt.to_period('M').astype(str)

# הגדרות מסלולים
TARIFF = 0.6
DISCOUNTS = {'הייטק': 0.10, 'לילה': 0.20, 'יום': 0.15, 'משפחה': 0.18, 'כללי': 0.07}
def is_hitech(hour): return hour >= 23 or hour < 17
def is_night(hour): return hour >= 23 or hour < 7
def is_day(hour, weekday): return weekday in range(0, 5) and 7 <= hour < 17
def is_family(hour, weekday): return weekday in range(0, 5) and 14 <= hour < 20

def classify_payment(row):
    hour = row['שעה']
    weekday = row['יום בשבוע']
    kwh = row['צריכה בקוט"ש']
    return pd.Series({
        'הייטק': kwh * TARIFF * (1 - DISCOUNTS['הייטק']) if is_hitech(hour) else kwh * TARIFF,
        'לילה': kwh * TARIFF * (1 - DISCOUNTS['לילה']) if is_night(hour) else kwh * TARIFF,
        'יום': kwh * TARIFF * (1 - DISCOUNTS['יום']) if is_day(hour, weekday) else kwh * TARIFF,
        'משפחה': kwh * TARIFF * (1 - DISCOUNTS['משפחה']) if is_family(hour, weekday) else kwh * TARIFF,
        'כללי': kwh * TARIFF * (1 - DISCOUNTS['כללי'])
    })

# ניתוח
payments = df.apply(classify_payment, axis=1)
payments['חודש'] = df['חודש']
monthly = payments.groupby('חודש').sum(numeric_only=True)
monthly_kwh = df.groupby('חודש')['צריכה בקוט"ש'].sum()
monthly['צריכה בקוט"ש'] = monthly_kwh
monthly['עלות רגילה (₪)'] = (monthly_kwh * TARIFF).round(2)

# תוצאה
monthly = monthly.reset_index().round(2)
monthly


,חודש,הייטק,לילה,יום,משפחה,כללי,"צריכה בקוט""ש",עלות רגילה (₪)
0,2024-04,17.51,17.66,18.13,18.12,17.53,31.42,18.85
1,2024-05,408.59,419.61,412.89,415.56,407.49,730.27,438.16
2,2024-06,672.78,680.80,694.46,690.19,668.52,1198.06,718.84
3,2024-07,662.58,663.32,680.97,673.35,659.57,1182.03,709.22
4,2024-08,778.62,792.83,799.14,802.43,776.97,1392.42,835.45
5,2024-09,688.05,692.69,711.73,707.83,684.32,1226.38,735.83
6,2024-10,296.75,302.47,302.96,304.33,297.16,532.55,319.53
7,2024-11,370.38,376.86,381.34,380.64,369.29,661.80,397.08
8,2024-12,522.00,522.35,537.05,537.17,516.53,925.69,555.41
9,2025-01,522.01,521.12,536.34,535.01,516.83,926.22,555.73
